## GoogLeNet V2 

GoogLeNet的第二版本的参考论文是:[Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift](https://arxiv.org/abs/1502.03167)

这个版本和上个版本的主要变化就是在每层激活函数之前加入批量归一化以加速训练过程，在使用了批量归一化以后，由于每层输入的数值稳定性增加(即分布大致相同，均值为0，方差为1)，因为我们可以使用较大的学习率，作者在实际实验过程中将GoogLeNet的第一个版本的0.0015的学习率调大了5倍以及30倍，并且不用Dropout，最终取得了比第一个版本更好的效果。

### 数据预处理

摘自[CS231n课程笔记翻译：神经网络笔记 2](https://zhuanlan.zhihu.com/p/21560667?refer=intelligentunit)

<img src="http://img.blog.csdn.net/20150313111616011?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvaGFwcHluZWFy/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/SouthEast" width="350">

数据预处理是在跑深度神经网络时必须的一个步骤，因为它可以加速收敛，甚至是一个简单的减去均值的操作都可以有比较明显的效果。一般来说，图像数据的每一维一般都是0-255之间的数字，因此数据点只会落在第一象限，而且图像数据具有很强的相关性，比如第一个灰度值为30，比较黑，那它旁边的一个像素值一般不会超过100，否则给人的感觉就像噪声一样。由于强相关性，数据点仅会落在第一象限的很小的区域中，形成类似上图所示的狭长分布。

而神经网络模型在初始化的时候，权重$W$是随机采样生成的，一个常见的神经元表示为：$ReLU(Wx+b) = max(Wx+b,0)$，即在$Wx+b=0$的两侧，对数据采用不同的操作方法。具体到$ReLU$就是一侧收缩，一侧保持不变。

随机的$Wx+b=0$表现为上图中的随机虚线，注意到，两条绿色虚线实际上并没有什么意义，在使用梯度下降时，可能需要很多次迭代才会使这些虚线对数据点进行有效的分割，就像紫色虚线那样，这势必会带来求解速率变慢的问题。更何况，我们这只是个二维的演示，数据占据四个象限中的一个，如果是几百、几千、上万维呢？而且数据在第一象限中也只是占了很小的一部分区域而已，可想而知不对数据进行预处理带来了多少运算资源的浪费，而且大量的数据外分割面在迭代时很可能会在刚进入数据中时就遇到了一个局部最优，导致$overfit$的问题。

这时，如果我们将数据减去其均值，数据点就不再只分布在第一象限，这时一个随机分界面落入数据分布的概率就增加了，而如果我们使用去除相关性的算法，例如$PCA$和$ZCA$白化，数据不再是一个狭长的分布，随机分界面有效的概率就又大大增加了。

不过计算协方差矩阵的特征值太耗时也太耗空间，我们一般最多只用到$z-score$处理(即每一维度减去自身均值，再除以自身标准差)，这样能使数据点在每维上具有相似的宽度，可以起到一定的增大数据分布范围，进而使更多随机分界面有意义的作用

### 数据预处理 - 均值减法

**均值减法**（Mean subtraction）是预处理最常用的形式。它对数据中每个独立特征减去平均值，从几何上可以理解为在每个维度上都将数据云的中心都迁移到原点。在numpy中，该操作可以通过代码$X -= np.mean(X, axis=0)$实现。而对于图像，更常用的是对所有像素都减去一个值，可以用$X -= np.mean(X)$实现，也可以在3个颜色通道上分别操作。

### 数据预处理 - 归一化

**归一化**（Normalization）是指将数据的所有维度都归一化，使其数值范围都近似相等。有两种常用方法可以实现归一化。第一种是先对数据做零中心化（zero-centered）处理，然后每个维度都除以其标准差，实现代码为$X /= np.std(X, axis=0)$。第二种方法是对每个维度都做归一化，使得每个维度的最大和最小值是1和-1。这个预处理操作只有在确信不同的输入特征有不同的数值范围（或计量单位）时才有意义，但要注意预处理操作的重要性几乎等同于学习算法本身。在图像处理中，由于像素的数值范围几乎是一致的（都在0-255之间），所以进行这个额外的预处理步骤并不是很必要。

<img src="https://pic2.zhimg.com/80/e743b6777775b1671c3b5503d7afbbc4_hd.jpg">  

一般数据预处理流程：**左边**：原始的2维输入数据。**中间**：在每个维度上都减去平均值后得到零中心化数据，现在数据云是以原点为中心的。**右边**：每个维度都除以其标准差来调整其数值范围。红色的线指出了数据各维度的数值范围，在中间的零中心化数据的数值范围不同，但在右边归一化数据中数值范围相同。

### 数据预处理 - PCA和白化（Whitening）

PCA和白化（Whitening）是另一种预处理形式。在这种处理中，先对数据进行零中心化处理，然后计算协方差矩阵，它展示了数据中的相关性结构。

```
# 假设输入数据矩阵X的尺寸为[N x D]
X -= np.mean(X, axis = 0) # 对数据进行零中心化(重要)
cov = np.dot(X.T, X) / X.shape[0] # 得到数据的协方差矩阵
```

数据协方差矩阵的第$(i, j)$个元素是数据第$i$个和第$j$个维度的协方差。具体来说，该矩阵的对角线上的元素是方差。还有，协方差矩阵是对称和半正定的。我们可以对数据协方差矩阵进行$SVD$（奇异值分解）运算。

```
U,S,V = np.linalg.svd(cov)
```

$U$的列是特征向量，$S$是装有奇异值的1维数组（因为$cov$是对称且半正定的，所以$S$中元素是特征值的平方）。为了去除数据相关性，将已经零中心化处理过的原始数据投影到特征基准上：

```
Xrot = np.dot(X,U) # 对数据去相关性
```

注意$U$的列是标准正交向量的集合（范式为1，列之间标准正交），所以可以把它们看做标准正交基向量。因此，投影对应$x$中的数据的一个旋转，旋转产生的结果就是新的特征向量。如果计算$Xrot$的协方差矩阵，将会看到它是对角对称的。$np.linalg.svd$的一个良好性质是在它的返回值U中，特征向量是按照特征值的大小排列的。我们可以利用这个性质来对数据降维，只要使用前面的小部分特征向量，丢弃掉那些包含的数据没有方差的维度。这个操作也被称为**<font color="blue">主成分分析（ Principal Component Analysis 简称PCA）降维**：

```
Xrot_reduced = np.dot(X, U[:,:100]) # Xrot_reduced 变成 [N x 100]
```

经过上面的操作，将原始的数据集的大小由$[N \times D]$降到了$[N \times 100]$，留下了数据中包含最大方差的100个维度。通常使用PCA降维过的数据训练线性分类器和神经网络会达到非常好的性能效果，同时还能节省时间和存储器空间。

最后一个在实践中会看见的变换是**<font color="blue">白化（whitening）**。白化操作的输入是特征基准上的数据，然后对每个维度除以其特征值来对数值范围进行归一化。该变换的几何解释是：如果数据服从多变量的高斯分布，那么经过白化后，数据的分布将会是一个均值为零，且协方差相等的矩阵。该操作的代码如下：

```
# 对数据进行白化操作:
# 除以特征值 
Xwhite = Xrot / np.sqrt(S + 1e-5)
```

使用白化时需要注意的是：夸大的噪声。注意分母中添加了1e-5（或一个更小的常量）来防止分母为0。该变换的一个缺陷是在变换的过程中可能会夸大数据中的噪声，这是因为它将所有维度都拉伸到相同的数值范围，这些维度中也包含了那些只有极少差异性(方差小)而大多是噪声的维度。在实际操作中，这个问题可以用更强的平滑来解决（例如：采用比1e-5更大的值）。

<img src="../img/Chapter4-Convolutional-Neural-Networks/4-3.jpg">

PCA/白化。**左边**：是二维的原始数据。**中间**：经过PCA操作的数据。可以看出数据首先是零中心的，然后变换到了数据协方差矩阵的基准轴上。这样就对数据进行了解相关（协方差矩阵变成对角阵）。**右边**：每个维度都被特征值调整数值范围，将数据协方差矩阵变为单位矩阵。从几何上看，就是对数据在各个方向上拉伸压缩，使之变成服从高斯分布的一个数据点分布。

**<font color="red">常见错误**。进行预处理很重要的一点是：任何预处理策略（比如数据均值）都只能在训练集数据上进行计算，算法训练完毕后再应用到验证集或者测试集上。例如，如果先计算整个数据集图像的平均值然后每张图片都减去平均值，最后将整个数据集分成训练/验证/测试集，那么这个做法是错误的。应该怎么做呢？应该先分成训练/验证/测试集，只是从训练集中求图片平均值，然后各个集（训练/验证/测试集）中的图像再减去这个平均值。
    
## Batch Normalization

摘自博文[Batch Normalization Accelerating Deep Network Training by Reducing Internal Covariate Shift》阅读笔记与实现](http://blog.csdn.net/happynear/article/details/44238541)

上一节我们讲到对输入数据进行预处理，减均值->$z-score$->白化可以逐级提升随机初始化的权重对数据分割的有效性，还可以降低overfit的可能性。我们都知道，现在的神经网络的层数都是很深的，如果我们对每一层的数据都进行处理，训练时间和overfit程度是否可以降低呢？Google的这篇论文给出了答案。

按照第前面说的理论，应当在每一层的激活函数之后，例如$ReLU=max(Wx+b,0)$之后，对数据进行归一化。然而，文章中说这样做在训练初期，分界面还在剧烈变化时，计算出的参数不稳定，所以退而求其次，在$Wx+b$之后进行归一化。因为初始的W是从标准高斯分布中采样得到的，而$W$中元素的数量远大于$x$，$Wx+b$每维的均值本身就接近0、方差接近1，所以在$Wx+b$后使用$Batch\,Normalization$能得到更稳定的结果。

文中使用了类似$z-score$的归一化方式：每一维度减去自身均值，再除以自身标准差，由于使用的是随机梯度下降法，这些均值和方差也只能在当前迭代的batch中计算，故作者给这个算法命名为$Batch Normalization$。这里有一点需要注意，像卷积层这样具有权值共享的层，$Wx+b$的均值和方差是对整张feature map求得的，在$batch\_size \times channel \times height \times width$这么大的一层中，对总共$batch\_size \times height \times width$个像素点统计得到一个均值和一个标准差，共得到$channel$组参数。

在Normalization完成后，Google的研究员仍对数值稳定性不放心，又加入了两个参数$\gamma$和$\beta$，使得

$$y^{(k)} = \gamma^{(k)} \hat{x}^{(k)} + \beta^{(k)}$$

注意到，如果我们令$\gamma$等于之前求得的标准差，$\beta$等于之前求得的均值，则这个变换就又将数据还原回去了。在他们的模型中，这两个参数与每层的$W$和$b$一样，是需要迭代求解的。

**算法优势**：

* 可以使用更高的学习率。如果每层的scale不一致，实际上每层需要的学习率是不一样的，同一层不同维度的scale往往也需要不同大小的学习率，通常需要使用最小的那个学习率才能保证损失函数有效下降，Batch Normalization将每层、每维的scale保持一致，那么我们就可以直接使用较高的学习率进行优化。

* 移除或使用较低的dropout。dropout是常用的防止overfitting的方法，而导致overfit的位置往往在数据边界处，如果初始化权重就已经落在数据内部，overfit现象就可以得到一定的缓解。论文中最后的模型分别使用10%、5%和0%的dropout训练模型，与之前的40%-50%相比，可以大大提高训练速度。

* 降低L2权重衰减系数。还是一样的问题，边界处的局部最优往往有几维的权重（斜率）较大，使用L2衰减可以缓解这一问题，现在用了Batch Normalization，就可以把这个值降低了，论文中降低为原来的5倍。

* 取消Local Response Normalization层。 由于使用了一种Normalization，再使用LRN就显得没那么必要了。而且LRN实际上也没那么work。

* 减少图像扭曲的使用。由于现在训练epoch数降低，所以要对输入数据少做一些扭曲，让神经网络多看看真实的数据。

### 总结

$Batch\,Normalization$的加速作用体现在两个方面：一是归一化了每层和每维度的scale，所以可以整体使用一个较高的学习率，而不必像以前那样迁就小scale的维度；二是归一化后使得更多的权重分界面落在了数据中，降低了overfit的可能性，因此一些防止overfit但会降低速度的方法，例如dropout和权重衰减就可以不使用或者降低其权重。

### GoogLeNet V2 网络参数

<img src="../img/Chapter4-Convolutional-Neural-Networks/4-4.png">

In [1]:
import mxnet as mx
import numpy as np

from mxnet import nd
from mxnet import gluon
from mxnet.gluon import nn
from mxnet import autograd

ctx = mx.gpu()
mx.random.seed(1)
import utils

In [2]:
class Conv_BN_ReLU(gluon.Block):
    def __init__(self, channels, kernel_size, strides=1, padding=0, verbose=False, **kwargs):
        super().__init__(**kwargs)
        self.verbose = verbose
        self.Conv2D = nn.Conv2D(channels, kernel_size=kernel_size, strides=strides, padding=padding)
        self.BN = nn.BatchNorm(axis=1)
        
    def forward(self, X):
        out = nd.relu(self.BN(self.Conv2D(X)))
        if self.verbose:
            print("Conv_BN_ReLU Shape : ", out.shape)
        return out

In [3]:
# test Conv_BN_ReLU
X = nd.random.uniform(shape=(64, 4, 32, 32), ctx=ctx)
cbr = Conv_BN_ReLU(64, kernel_size=3, padding=1)
cbr.initialize(ctx=ctx)

In [4]:
class InceptionPT(gluon.Block):
    def __init__(self, n1_1, n2_1, n2_3, n3_1, n3_double_3, n4_1, debug=False, **kwargs):
        super().__init__(**kwargs)
        
        self.debug = debug
        # path 1
        self.p2_conv_1 = Conv_BN_ReLU(n2_1, kernel_size=1)
        self.p2_conv_2 = Conv_BN_ReLU(n2_3, kernel_size=3, padding=1)
        
        # path 2
        self.p3_conv_1 = Conv_BN_ReLU(n3_1, kernel_size=1)
        self.p3_conv_3_1 = Conv_BN_ReLU(n3_double_3, kernel_size=3, padding=1)
        self.p3_conv_3_2 = Conv_BN_ReLU(n3_double_3, kernel_size=3, padding=1)
        
        # max pooling strides=2
        self.p4_pool_3 = nn.MaxPool2D(pool_size=3, strides=1, padding=1)

        # pass through
        self.pass_through = nn.MaxPool2D(pool_size=2, strides=2)
    
    def forward(self, X):
        p2 = self.pass_through(self.p2_conv_2(self.p2_conv_1(X)))
        p3 = self.pass_through(self.p3_conv_3_2(self.p3_conv_3_1(self.p3_conv_1(X))))
        p4 = self.pass_through(self.p4_pool_3(X))
        
        if self.debug:
            print("p2 out shape : ", p2.shape)
            print("p3 out shape : ", p3.shape)
            print("p4 out shape : ", p4.shape)

        return nd.concat(p2, p3, p4, dim=1)

In [5]:
# test InceptionPT
mph =InceptionPT(0, 128, 192, 192, 256, 0)
X = nd.random.uniform(shape=(64, 576, 14, 14), ctx=ctx)
mph.initialize(ctx=ctx)
mph(X).shape

(64, 1024, 7, 7)

In [6]:
# 4条path
class InceptionV2(gluon.Block):
    def __init__(self, n1_1, n2_1, n2_3, n3_1, n3_double_3, n4_1, pool_type='max', debug=False, **kwargs):
        super().__init__(**kwargs)
        self.debug = debug
        self.n1_1 = n1_1
        # path 1 1X1 Convolution
        if n1_1:
            self.p1_conv_1 = Conv_BN_ReLU(n1_1, kernel_size=1)
            
        # path 2 3X3 Convolution
        self.p2_conv_1 = Conv_BN_ReLU(n2_1, kernel_size=1)
        self.p2_conv_3 = Conv_BN_ReLU(n2_3, kernel_size=3, padding=1)
        
        # path 3 double 3X3 Convolution
        self.p3_conv_1 = Conv_BN_ReLU(n3_1, kernel_size=1)
        self.p3_conv_3_1 = Conv_BN_ReLU(n3_double_3, kernel_size=3, padding=1)
        self.p3_conv_3_2 = Conv_BN_ReLU(n3_double_3, kernel_size=3, padding=1)
        
        # path 4 Max/Avg pooling + 1X1 conv
        if pool_type == 'max':
            self.p4_pool_3 = nn.MaxPool2D(pool_size=3, padding=1, strides=1)
        elif pool_type == 'avg':
            self.p4_pool_3 = nn.AvgPool2D(pool_size=3, padding=1, strides=1)
        else:
            raise mx.MXNetError("Unknown pooling type.")
        self.p4_conv_1 = Conv_BN_ReLU(n4_1, kernel_size=1)
        
        # 
        
    def forward(self, X):
        if self.n1_1:
            p1 = self.p1_conv_1(X)
        p2 = self.p2_conv_3(self.p2_conv_1(X))
        p3 = self.p3_conv_3_2(self.p3_conv_3_1(self.p3_conv_1(X)))
        p4 = self.p4_conv_1(self.p4_pool_3(X))
        
        if self.debug:
            if self.n1_1:
                print("p1 out shape : ", p1.shape)
            print("p2 out shape : ", p2.shape)
            print("p3 out shape : ", p3.shape)
            print("p4 out shape : ", p4.shape)
        
        if self.n1_1:
            return nd.concat(p1, p2, p3, p4, dim=1)
        else:
            return nd.concat(p2, p3, p4, dim=1)

In [7]:
# test inceptionV2
incep = InceptionV2(192, 96, 128, 96, 128, 128, pool_type='avg')
incep.initialize(ctx=ctx)

X = nd.random.uniform(shape=(32, 576, 14, 14), ctx=ctx)
incep(X).shape

(32, 576, 14, 14)

In [8]:
class GoogLeNetV2(gluon.Block):
    def __init__(self, num_classes, verbose=False, **kwargs):
        super().__init__(**kwargs)
        
        self.verbose = verbose
        with self.name_scope():
            # block 1
            b1 = nn.Sequential()
            b1.add(
                # 这个padding是算出来的(224-7+2*p)/2 + 1 = 112 -> p=3
                Conv_BN_ReLU(channels=64, kernel_size=7, strides=2, padding=3), 
                # 这个padding是算出来的(112-3+2*p)/2 + 1 = 56 -> p=1
                nn.MaxPool2D(pool_size=3, strides=2, padding=1)
            )
            # block 2
            b2 = nn.Sequential()
            b2.add(
                Conv_BN_ReLU(channels=192, kernel_size=3, padding=1),
                # 这个padding是算出来的(56-3+2*p)/2 + 1 = 28 -> p=1
                nn.MaxPool2D(pool_size=3, strides=2, padding=1)
            )
            # block 3
            b3 = nn.Sequential()
            b3.add(
                InceptionV2(64, 64, 64, 64, 96, 32, pool_type='avg'),
                InceptionV2(64, 64, 96, 64, 96, 64, pool_type='avg'),
                InceptionPT(0, 128, 160, 64, 96, 0) 
            )
            b4 = nn.Sequential()
            b4.add(
                InceptionV2(224, 64, 96, 96, 128, 128, pool_type='avg'),
                InceptionV2(192, 96, 128, 96, 128, 128, pool_type='avg'),
                InceptionV2(160, 128, 160, 128, 160, 128, pool_type='avg'),
                InceptionV2(96, 128, 192, 160, 192, 128, pool_type='avg'),
                InceptionPT(0, 128, 192, 192, 256, 0) 
            )
            b5 = nn.Sequential()
            b5.add(
                InceptionV2(352, 192, 320, 160, 224, 128, pool_type='avg'),
                InceptionV2(352, 192, 320, 192, 224, 128, pool_type='max'),
                nn.AvgPool2D(pool_size=7, strides=1)
            )
            b6 = nn.Sequential()
            b6.add(
                nn.Flatten(),
                nn.Dense(num_classes)
            )
            self.net = nn.Sequential()
            self.net.add(b1, b2, b3, b4, b5, b6)

    def forward(self, X):
        out = X
        for i, blk in enumerate(self.net):
            out = blk(out)
            if self.verbose:
                print("blk%d output : %s" % (i+1, out.shape))
        return out

In [9]:
google_net = GoogLeNetV2(10, verbose=True)
google_net.initialize(ctx=ctx)
X = nd.random.uniform(shape=(32, 3, 224, 224), ctx=ctx)
y = google_net(X)

blk1 output : (32, 64, 56, 56)
blk2 output : (32, 192, 28, 28)
blk3 output : (32, 576, 14, 14)
blk4 output : (32, 1056, 7, 7)
blk5 output : (32, 1024, 1, 1)
blk6 output : (32, 10)


In [10]:
from time import time

batch_size = 64
train_data, test_data = utils.load_dataset(batch_size, resize=224, data_type='cifar10')

gln = GoogLeNetV2(num_classes=10, verbose=False)
gln.collect_params().initialize(mx.init.Xavier(), ctx=ctx, force_reinit=True)

softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(gln.collect_params(), 'sgd', {'learning_rate' : 0.5})

epochs = 20

niter = 0
moving_loss = 0.0
smoothing_constant = 0.9

from time import time
for epoch in range(epochs):
    start = time()
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        with autograd.record():
            output = gln(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(batch_size)
        
        niter += 1
        curr_loss = nd.mean(loss).asscalar()
        moving_loss = smoothing_constant * moving_loss + (1-smoothing_constant) * curr_loss
        estimated_loss = moving_loss / (1 - smoothing_constant**niter)
    
    train_acc = utils.evaluate_accuracy_gluon(train_data, gln, ctx)
    test_acc = utils.evaluate_accuracy_gluon(test_data, gln, ctx)
    print("Epoch %d, Moving Train Avg loss %.5f, Train acc %.5f, Test acc %.5f, Time consume %.5f s."
         % (epoch, estimated_loss, train_acc, test_acc, time() - start))

Epoch 0, Moving Train Avg loss 1.28596, Train acc 0.49908, Test acc 0.49460, Time consume 295.52580 s.
Epoch 1, Moving Train Avg loss 0.92046, Train acc 0.68216, Test acc 0.66370, Time consume 293.11788 s.
Epoch 2, Moving Train Avg loss 0.77204, Train acc 0.68622, Test acc 0.66320, Time consume 293.95373 s.
Epoch 3, Moving Train Avg loss 0.67349, Train acc 0.76484, Test acc 0.72480, Time consume 294.15910 s.
Epoch 4, Moving Train Avg loss 0.55544, Train acc 0.85008, Test acc 0.79210, Time consume 294.59429 s.
Epoch 5, Moving Train Avg loss 0.36633, Train acc 0.84360, Test acc 0.77780, Time consume 293.18314 s.
Epoch 6, Moving Train Avg loss 0.32562, Train acc 0.87646, Test acc 0.80130, Time consume 292.95229 s.
Epoch 7, Moving Train Avg loss 0.33631, Train acc 0.90062, Test acc 0.81400, Time consume 292.91839 s.
Epoch 8, Moving Train Avg loss 0.23266, Train acc 0.94068, Test acc 0.83530, Time consume 292.92914 s.
Epoch 9, Moving Train Avg loss 0.25051, Train acc 0.95532, Test acc 0.847